In [1]:
import numpy as np
import time
from numpy import ones,vstack
from numpy.linalg import lstsq

import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from random import random, randint, uniform

import os
import shutil

from statistics import median
import pickle

In [2]:
def generate_decoding_domain(observations, radius=10, time_precision=0.005):
    
    pddl_str = "(define (domain flight)\n"
    pddl_str += "\n"
    
    # Predicates
    pddl_str += "(:predicates\n"
    pddl_str += "\t;HS locations\n"
    pddl_str += "\t(flying-straight)\n"
    pddl_str += "\t(adjusting-left)\n"
    pddl_str += "\t(adjusting-right)\n"
    pddl_str += "\n"    
    pddl_str += "\t;monitor locations\n"
    for i in range(len(observations)+1):
        pddl_str += "\t(monitor_{})\n".format(i)
    pddl_str += ")\n"
    pddl_str += "\n"
    
    # Functions
    pddl_str += "(:functions\n"
    pddl_str += "\t;HS variables\n"
    pddl_str += "\t(x1) ; coordinate x\n"
    pddl_str += "\t(x2) ; coordinate y\n"
    pddl_str += "\t(theta) ; orientation\n"
    pddl_str += "\t(v) ; linear speed\n"
    pddl_str += "\t(w) ; angular speed\n"
    pddl_str += "\n"
    
   
    pddl_str += "\t;monitor variables\n"
    pddl_str += "\t(running_time)  ;; time\n"
    pddl_str += "\n"
    
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-flying-straight\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-adjusting-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t\t(increase (theta) (* #t 0.31415927))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-adjusting-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x1) (* #t (* (v) (cos (theta)))))\n"
    pddl_str += "\t\t(increase (x2) (* #t (* (v) (sin (theta)))))\n"
    pddl_str += "\t\t(increase (theta) (* #t -0.31415927))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    
    # HS Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:action stop-adjust-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-left))\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action stop-adjust-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-right))\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action adjust-left\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (flying-straight))\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action adjust-right\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(flying-straight)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (flying-straight))\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action toggle-adjust-LR\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-left))\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:action toggle-adjust-LR\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(adjusting-right)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (adjusting-right))\n"
    pddl_str += "\t\t(adjusting-left)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Invariants
    
    pddl_str += ";;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Invariants\n"
    pddl_str += ";;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
             
    
    # Monitor Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    
    for i in range(len(observations)+1):
        pddl_str += "(:process flow-monitor_{}\n".format(i)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(increase (running_time) (* #t 1.0))\n"
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
        
    # Monitor Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
        
    for i in range(len(observations)):
        obs = observations[i]
        
        pddl_str += "(:event validate_{}\n".format(i+1)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t\t(<= {} running_time) ; t_{} - 0.05\n".format(obs[2] - time_precision, i+1)
        pddl_str += "\t\t(> {} running_time) ; t_{} + 0.05\n".format(obs[2] + time_precision, i+1)
        pddl_str += "\t\t(< (+ (^ (- x1 {}) 2) (^ (- x2 {}) 2)) (^ {} 2) ) ; \phi_{}\n".format(round(obs[0],3), round(obs[1],3), radius, i+1)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(not (monitor_{}))\n".format(i)
        pddl_str += "\t\t(monitor_{})\n".format(i+1)
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
       
        
    pddl_str += ")"
    
    
    return pddl_str


In [3]:
def generate_decoding_problem(observations, theta, v):
    
    pddl_str = "(define (problem problem_name) (:domain platoon)\n"
    pddl_str += "\n"

    # Initial state
    pddl_str += "(:init\n"
    pddl_str += "\t; HS initial state\n"
    pddl_str += "\t(= (x1) 0)\n"
    pddl_str += "\t(= (x2) 0)\n"
    pddl_str += "\t(= (theta) {})\n".format(theta)
    pddl_str += "\t(= (v) {})\n".format(v)
    pddl_str += "\t(flying-straight)\n"
    pddl_str += "\t; monitor initial state\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\t(monitor_0)\n"
    pddl_str += "\n"
    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(monitor_{})\n".format(len(observations))
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str

In [4]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/flight/ENHSP_e"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)

with open("flight.pkl", "rb") as f:
    instances = pickle.load(f)

for pnum in range(num_instances):
    for h in horizons:
        dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        pname = 'inst_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        
        
        domain_str = generate_decoding_domain(instances[pnum]["observations"][:h])
        with open(dfolder + '/' + dname, 'w') as f:
            f.write(domain_str)
            
        problem_str = generate_decoding_problem(instances[pnum]["observations"][:h], instances[pnum]["init_theta"], instances[pnum]["init_v"])
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(problem_str)

In [5]:
#horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#dfolder = "benchmarks/flight/HSE_Pe"

times = {}
instances = list(range(num_instances))
for h in horizons:
    h_times = []
    working_instances = []
    print(instances)
    for pnum in instances:  
        
        try:
            dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
            pname = 'inst_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)

            cmd = "timeout 300 enhsp_exp -o {dfolder}/{dname} -f {dfolder}/{pname} -dp 0.01 -dh 0.01 -de 0.01 -dv 0.01 -s WAStar -h hmrp -pt -sp {dfolder}/plan_{pnum}_{h}".format(dfolder=dfolder, dname=dname, pname=pname, pnum=pnum, h=h)

            t = time.time()
            os.system(cmd)
            elapsed = time.time() - t

            shutil.move(dfolder + "/{pname}_search_WAStar_h_hmrp_break_ties_arbitrary.npt".format(pname=pname), dfolder + "/solution_{pnum}_{h}.npt".format(pnum=pnum, h=h))
            h_times += [elapsed]
            working_instances += [pnum]
        except:
            print("instance {} failed".format(pnum))
        
    times[h] = h_times
    print(h, h_times)
    
    instances = working_instances
    
with open("{}/enhsp_e_001".format(dfolder), "w") as f:
    for k,h_times in times.items():
        f.write(" ".join(map(str, h_times)) + "\n")


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Domain parsed
Problem parsed
Grounding..
Grounding Time: 19
Aibr Preprocessing
|F|:6
|X|:4
Aibr Preprocessing
|A|:5
|P|:6
|E|:2
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 8
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=203.0
f(n) = 203.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.004) Frontier Size: 0
Extracting plan with execution delta: 0.01
Plan under Validation/Simulation: [(0,(------>waiting)), (0.01,(------>waiting)), (0.02,(------>waiting)), (0.03,(------>waiting)), (0.04,(------>waiting)), (0.05,(------>waiting)), (0.06,(------>waiting)), (0.07,(------>waiting)), (0.08,(------>waiting)), (0.09,(------>waiting)), (0.10,(------>waiting)), (0.11,(------>waiting)), (0.12,(------>waiting)), (0.13,(------>waiting)), (0.14,(------>waiting)), (0.15,(------>waiting)), (0.16,(------>waiting)), (0.17,(------>waiting)), (0.18,(------>wai

Aibr Preprocessing
|A|:5
|P|:6
|E|:2
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 6
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=203.0
f(n) = 203.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.002) Frontier Size: 0
f(n) = 204.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 0.221) Frontier Size: 15349
Extracting plan with execution delta: 0.01
Plan under Validation/Simulation: [(0,(------>waiting)), (0.01,(------>waiting)), (0.02,(------>waiting)), (0.03,(------>waiting)), (0.04,(------>waiting)), (0.05,(------>waiting)), (0.06,(------>waiting)), (0.07,(------>waiting)), (0.08,(------>waiting)), (0.09,(------>waiting)), (0.10,(------>waiting)), (0.11,(------>waiting)), (0.12,(------>waiting)), (0.13,(------>waiting)), (0.14,(------>waiting)), (0.15,(------>waiting)), (0.16,(------>waiting)), (0.17,(------>waiting)), (0.18,(------>waiting)), (0.19,(------>wait

Domain parsed
Problem parsed
Grounding..
Grounding Time: 9
Aibr Preprocessing
|F|:6
|X|:4
Aibr Preprocessing
|A|:5
|P|:6
|E|:2
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 6
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=203.0
f(n) = 203.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.002) Frontier Size: 0
f(n) = 204.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 0.214) Frontier Size: 15349
Extracting plan with execution delta: 0.01
Plan under Validation/Simulation: [(0,(------>waiting)), (0.01,(------>waiting)), (0.02,(------>waiting)), (0.03,(------>waiting)), (0.04,(------>waiting)), (0.05,(------>waiting)), (0.06,(------>waiting)), (0.07,(------>waiting)), (0.08,(------>waiting)), (0.09,(------>waiting)), (0.10,(------>waiting)), (0.11,(------>waiting)), (0.12,(------>waiting)), (0.13,(------>waiting)), (0.14,(------>waiting)), (0.15,(------>waiting)), (0.

Domain parsed
Problem parsed
Grounding..
Grounding Time: 9
Aibr Preprocessing
|F|:6
|X|:4
Aibr Preprocessing
|A|:5
|P|:6
|E|:2
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 8
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=203.0
f(n) = 203.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.002) Frontier Size: 0
f(n) = 204.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 0.242) Frontier Size: 15349
Extracting plan with execution delta: 0.01
Plan under Validation/Simulation: [(0,(------>waiting)), (0.01,(------>waiting)), (0.02,(------>waiting)), (0.03,(------>waiting)), (0.04,(------>waiting)), (0.05,(------>waiting)), (0.06,(------>waiting)), (0.07,(------>waiting)), (0.08,(------>waiting)), (0.09,(------>waiting)), (0.10,(------>waiting)), (0.11,(------>waiting)), (0.12,(------>waiting)), (0.13,(------>waiting)), (0.14,(------>waiting)), (0.15,(------>waiting)), (0.

Domain parsed
Problem parsed
Grounding..
Grounding Time: 9
Aibr Preprocessing
|F|:6
|X|:4
Aibr Preprocessing
|A|:5
|P|:6
|E|:2
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 5
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=203.0
f(n) = 203.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.005) Frontier Size: 0
f(n) = 204.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 0.212) Frontier Size: 15349
Extracting plan with execution delta: 0.01
Plan under Validation/Simulation: [(0,(------>waiting)), (0.01,(------>waiting)), (0.02,(------>waiting)), (0.03,(------>waiting)), (0.04,(------>waiting)), (0.05,(------>waiting)), (0.06,(------>waiting)), (0.07,(------>waiting)), (0.08,(------>waiting)), (0.09,(------>waiting)), (0.10,(------>waiting)), (0.11,(------>waiting)), (0.12,(------>waiting)), (0.13,(------>waiting)), (0.14,(------>waiting)), (0.15,(------>waiting)), (0.

...

Domain parsed
Problem parsed
Grounding..
Grounding Time: 17
Aibr Preprocessing
|F|:24
|X|:4
Aibr Preprocessing
|A|:5
|P|:24
|E|:20
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 6
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=2021.0
f(n) = 2021.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.007) Frontier Size: 0
f(n) = 2022.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 0.999) Frontier Size: 15349
-------------Time: 10s ; Expanded Nodes: 138552 (Avg-Speed 13855.0 n/s); Evaluated States: 322339
-------------Time: 20s ; Expanded Nodes: 337761 (Avg-Speed 16888.0 n/s); Evaluated States: 795135
-------------Time: 30s ; Expanded Nodes: 534192 (Avg-Speed 17806.0 n/s); Evaluated States: 1317173
-------------Time: 40s ; Expanded Nodes: 718049 (Avg-Speed 17951.0 n/s); Evaluated States: 1806162
-------------Time: 50s ; Expanded Nodes: 942920 (Avg-Speed 18858.0 n/s); Eval

Domain parsed
Problem parsed
Grounding..
Grounding Time: 16
Aibr Preprocessing
|F|:24
|X|:4
Aibr Preprocessing
|A|:5
|P|:24
|E|:20
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 7
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=2021.0
f(n) = 2021.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.006) Frontier Size: 0
-------------Time: 10s ; Expanded Nodes: 156780 (Avg-Speed 15678.0 n/s); Evaluated States: 380992
-------------Time: 20s ; Expanded Nodes: 337831 (Avg-Speed 16891.0 n/s); Evaluated States: 813400
-------------Time: 30s ; Expanded Nodes: 524413 (Avg-Speed 17480.0 n/s); Evaluated States: 1278923
-------------Time: 40s ; Expanded Nodes: 709972 (Avg-Speed 17749.0 n/s); Evaluated States: 1768862
-------------Time: 50s ; Expanded Nodes: 933151 (Avg-Speed 18663.0 n/s); Evaluated States: 2315193
-------------Time: 60s ; Expanded Nodes: 1151765 (Avg-Speed 19196.0 n/s); E

Domain parsed
Problem parsed
Grounding..
Grounding Time: 21
Aibr Preprocessing
|F|:34
|X|:4
Aibr Preprocessing
|A|:5
|P|:34
|E|:30
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 7
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=3031.0
f(n) = 3031.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.008) Frontier Size: 0
f(n) = 3032.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 1.519) Frontier Size: 15349
-------------Time: 10s ; Expanded Nodes: 88639 (Avg-Speed 8863.0 n/s); Evaluated States: 195669
-------------Time: 20s ; Expanded Nodes: 187186 (Avg-Speed 9359.0 n/s); Evaluated States: 428291
-------------Time: 30s ; Expanded Nodes: 306381 (Avg-Speed 10212.0 n/s); Evaluated States: 712856
-------------Time: 40s ; Expanded Nodes: 417198 (Avg-Speed 10429.0 n/s); Evaluated States: 1005467
-------------Time: 50s ; Expanded Nodes: 528472 (Avg-Speed 10569.0 n/s); Evaluate

Domain parsed
Problem parsed
Grounding..
Grounding Time: 37
Aibr Preprocessing
|F|:44
|X|:4
Aibr Preprocessing
|A|:5
|P|:44
|E|:40
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 13
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=4041.0
f(n) = 4041.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.011) Frontier Size: 0
f(n) = 4042.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 2.054) Frontier Size: 15349
-------------Time: 10s ; Expanded Nodes: 62008 (Avg-Speed 6200.0 n/s); Evaluated States: 142150
-------------Time: 20s ; Expanded Nodes: 140579 (Avg-Speed 7028.0 n/s); Evaluated States: 317828
-------------Time: 30s ; Expanded Nodes: 226472 (Avg-Speed 7549.0 n/s); Evaluated States: 502873
-------------Time: 40s ; Expanded Nodes: 301495 (Avg-Speed 7537.0 n/s); Evaluated States: 690439
-------------Time: 50s ; Expanded Nodes: 374606 (Avg-Speed 7492.0 n/s); Evaluated S

Domain parsed
Problem parsed
Grounding..
Grounding Time: 38
Aibr Preprocessing
|F|:44
|X|:4
Aibr Preprocessing
|A|:5
|P|:44
|E|:40
Delta time heuristic model:0.01
Delta time planning model:0.01
Delta time search-execution model:0.01
Delta time validation model:0.01
H1 Setup Time (msec): 7
Setting horizon to:NaN
Running WA-STAR
h(n = s_0)=4041.0
f(n) = 4041.0 (Expanded Nodes: 0, Evaluated States: 0, Time: 0.009) Frontier Size: 0
f(n) = 4042.0 (Expanded Nodes: 10503, Evaluated States: 26055, Time: 2.167) Frontier Size: 15349
-------------Time: 10s ; Expanded Nodes: 58248 (Avg-Speed 5824.0 n/s); Evaluated States: 132558
-------------Time: 20s ; Expanded Nodes: 118627 (Avg-Speed 5931.0 n/s); Evaluated States: 270252
-------------Time: 30s ; Expanded Nodes: 189328 (Avg-Speed 6310.0 n/s); Evaluated States: 433644
-------------Time: 40s ; Expanded Nodes: 265991 (Avg-Speed 6649.0 n/s); Evaluated States: 613344
-------------Time: 50s ; Expanded Nodes: 339180 (Avg-Speed 6783.0 n/s); Evaluated St